# Maze and path finding

For searching, we have Depth-First Search, Breadth-First Search, Dijkstra, Best-First Search and A\* algorithms.

Using searching to solve a problem, we should know the target doesn't only means a specific location on the graph, but also can be a state reached during the searching process. Which means even you work on the same location, but it's not the same node in abstraction since you are in different state.

In [36]:
import collections
import heapq
class Solution(object):
    def shortestPathAllKeys(self, grid):
        M = len(grid)
        N = len(grid[0])
        
        location = {v: (m, n) 
                    for m, row in enumerate(grid)
                    for n, v in enumerate(row) 
                    if v not in '.#'}
    
        def neighbors(m, n):
            for nm, nn in ((m-1, n), (m+1, n), (m, n-1), (m, n+1)):
                if 0<=nm<M and 0<=nn<N:
                    yield nm, nn
    
        def bfs_from(source):
            m, n = location[source]
            visited = [[False]*N for _ in range(M)]
            visited[m][n] = True
            queue = [(m, n, 0)]
            dist = {}
            while queue:
                m, n, d = queue.pop(0)
                if grid[m][n] != source and grid[m][n]!= '.':
                    dist[grid[m][n]] = d
                    continue # We don't want a path containing two points
                for nm, nn in neighbors(m, n):
                    if grid[nm][nn] != '#' and not visited[nm][nn]:
                        visited[nm][nn] = True
                        queue.append((nm, nn, d+1))
            return dist
        dists = {place: bfs_from(place) for place in location}
        target_state = 2**sum(p.islower() for p in location)-1
        pq = [(0, '@', 0)]
        cost_sofar = collections.defaultdict(lambda: float('inf'))
        cost_sofar['@', 0] = 0
        while pq:
            d, place, state = heapq.heappop(pq)
            if cost_sofar[place, state] < d: 
                continue
            if state == target_state:
                return d
            
            for nextstation in dists[place]:
                nextd = dists[place][nextstation]
                tempstate = state
                if nextstation.islower():
                    tempstate |= (1<< (ord(nextstation)-ord('a')))
                elif nextstation.isupper():
                    if not (state & (1<< (ord(nextstation)-ord('A')))):
                        continue
                
                if d+nextd < cost_sofar[nextstation, tempstate]:
                    cost_sofar[nextstation, tempstate] = d+nextd
                    heapq.heappush(pq, (d+nextd, nextstation, tempstate))
        return -1
                
    

In [37]:
s = Solution()

In [38]:
grid = ["@.a.#","###.#","b.A.B"]
s.shortestPathAllKeys(grid)

8

Example [Link](https://leetcode.com/problems/longest-increasing-path-in-a-matrix/description/):

In [4]:
class Solution(object):
    def longestIncreasingPath(self, matrix):
        """
        :type matrix: List[List[int]]
        :rtype: int
        """
        if not matrix:
            return 0
        M = len(matrix)
        N = len(matrix[0])
        adj = [[[] for _ in range(N)] for _ in range(M)]
        for m in range(M):
            for n in range(N):
                for i, j in ((m+1, n), (m, n+1)):
                    if i<M and j<N:
                        if matrix[i][j]> matrix[m][n]:
                            adj[m][n].append((i,j))
                        elif matrix[i][j] < matrix[m][n]:
                            adj[i][j].append((m,n))
        state = [[0]*N for _ in range(M)]
        number = [[1]*N for _ in range(M)]
        maxnum = 1
        stack= [(0,0)]
        while stack:
            while stack:
                m, n = stack[-1]
                v = matrix[m][n]
                s = state[m][n]
                if s == len(adj[m][n]):
                    if s > 0:
                        i,j = adj[m][n][-1]
                        if number[i][j]+1> number[m][n]:
                            number[m][n] = number[i][j]+1
                        if number[m][n] > maxnum:
                            maxnum = number[m][n]
                    stack.pop()
                else:
                    while True:
                        if s > 0:
                            pi,pj = adj[m][n][s-1]
                            if number[pi][pj]+1> number[m][n]:
                                number[m][n] = number[pi][pj]+1
                        i, j = adj[m][n][s]
                        state[m][n] += 1
                        s = state[m][n]
                        if state[i][j] < len(adj[i][j]):
                            stack.append((i, j))
                            break
                        if state[m][n] == len(adj[m][n]):
                            break
            for nid in range(M*N):
                curm = int(nid/N)
                curn = int(nid%N)
                if state[curm][curn] < len(adj[curm][curn]):
                    stack.append((curm, curn))
                    break
        return maxnum

In [5]:
s=Solution()

In [6]:
matrix = [[7,7,5],[2,4,6],[8,2,0]]
s.longestIncreasingPath(matrix)

4

A recursive version:

In [1]:
class Solution(object):
    def longestIncreasingPath(self, matrix):
        """
        :type matrix: List[List[int]]
        :rtype: int
        """
        if not matrix:
            return 0
        self.matrix = matrix
        M = len(matrix)
        self.M = M
        N = len(matrix[0])
        self.N = N
        self.adj = [[[] for _ in range(N)] for _ in range(M)]
        for m in range(M):
            for n in range(N):
                for i, j in ((m+1, n), (m, n+1)):
                    if i<M and j<N:
                        if matrix[i][j]> matrix[m][n]:
                            self.adj[m][n].append((i,j))
                        elif matrix[i][j] < matrix[m][n]:
                            self.adj[i][j].append((m,n))
        self.number = [[0]*N for _ in range(M)]
        maxnum = 1
        for m in range(M):
            for n in range(N):
                length = self.dfs(m, n)
                maxnum = max(maxnum, length)
        return maxnum
    
    def dfs(self, m, n):
        if self.number[m][n] > 0:
            return self.number[m][n]
        maxn = 1
        for i, j in self.adj[m][n]:
            length = 1+self.dfs(i, j)
            maxn = max(maxn, length)
        self.number[m][n] = maxn
        return maxn

In [2]:
s=Solution()

In [3]:
matrix = [[7,7,5],[2,4,6],[8,2,0]]
s.longestIncreasingPath(matrix)

4

**Summay**

A comparison between recursive version and the iteration version equiped with the a stack.
It doesn't means recursive will be slower all the time, these code for this problem is a good example.

In python, we may spend some time unintensionally revisiting some nodes, which can be avoid during the recursive version. We need to realize that this is also where we complain about for using recursive algorithm, but if we can solve the problem of revising some nodes in recursive, like here using extra memory, using recursive may be even more convinent and faster.